# 오늘의 영어 단어 생성하기

In [1]:
import json

from tqdm.notebook import tqdm
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import HumanMessage, AIMessage
import pandas as pd

In [2]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=1.0)

In [3]:
class Word(BaseModel):
    word: str = Field("영어 단어")
    imoj: str = Field("word에 어울리는 적절한 이모지")
    meaning: str = Field("word의 의미, 뜻 한국어로")

In [4]:
parser = JsonOutputParser(pydantic_object=Word)

In [5]:
format_instruction = parser.get_format_instructions()

In [6]:
fewshot = [
    HumanMessage(content=f"영어 단어 하나와 그 단어에 어울리는 이모지와 그 뜻을 만들어줘. 포맷은 다음을 따라 만들어줘\n{format_instruction}"),
    AIMessage(content='{"word": "situation", "imoj": "🤔", "meaning": "situation은 어떤 일이 일어나고 있는 특정한 장소나 시간을 말합니다. 또한 그것은 어떤 것의 현재 상태나 조건을 가리킬 수도 있습니다. situation은 문제나 어려움이 발생한 상황을 강조하기 위해 사용될 수 있습니다."}'),
    HumanMessage(content=f"영어 단어 하나와 그 단어에 어울리는 이모지와 그 뜻을 만들어줘. 포맷은 다음을 따라 만들어줘\n{format_instruction}"),
    AIMessage(content='{"word": "concern", "imoj": "😟", "meaning": "concern은 걱정이나 근심을 의미합니다. 그것은 누군가에게 감정적으로나 육체적으로 영향을 미칠 수 있는 무언가에 대한 관심이나 걱정을 나타낼 수 있습니다. concern은 또한 중요성이나 관련성을 의미할 수도 있습니다."}'),
    HumanMessage(content=f"영어 단어 하나와 그 단어에 어울리는 이모지와 그 뜻을 만들어줘. 포맷은 다음을 따라 만들어줘\n{format_instruction}"),
]

In [7]:
chain = model | parser

In [8]:
chain.invoke(fewshot)

{'word': 'love',
 'imoj': '❤️',
 'meaning': 'love는 강한 감정적인 애착이나 애정을 의미합니다. 그것은 다른 사람이나 사물에 대한 열정적인 애정이나 관심을 나타낼 수 있습니다. love는 또한 친절하고 배려심 있는 행동, 상대에 대한 관심과 존경을 포함할 수도 있습니다.'}

In [9]:
sample_list = []
n_sample = 10
for _ in tqdm(range(10), total=n_sample):
    sample = chain.invoke(fewshot)
    sample_list.append(sample)
    

In [10]:
df = pd.DataFrame(sample_list)

In [11]:
df.to_excel("./words.xlsx", index=False)